Introduction/Business Problem:

A contractor wants to open a gym somewhere in Denver, Colorado. She is looking for areas where there is not a lot of gyms around and preferebly a place with low gym rating.
The purpose of the gym is to be more accessable to people of that area and provide a good service to its customers.

This notebook will look into different neighbourhoods in the Denver area and pick the most suitable location base don the customer's nead. This note book will come with visualization tools to help the customer understand their business competition and how to gain a competative advantage.

Data:

The data that will be used to solve the business problem is the ratings and locations of each gym in the city of Denver. Each neighbourhood will be segmented and clustered to gain insight on the performance of the gyms in the Denver area. 

We will start by aquairing the data from Foursquare then change/wrangle the data to fit our analysis. The data will put into a pandas dataframe that includes some of the following as a minimum: neighbourhoods, latitude and longitude. all unessesary data will be dropped from the data set. 

Afterwards, we will look into the venues in each neighbourhood and extract the gyms in the area, find their rating and produce a detailed report with our recommendation on which neighbourhood is best suited for a new gym.

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print('Libraries imported.')

Libraries imported.


In [2]:
denver_data = pd.read_csv('Denver_Neighborhoods.csv', header=None)

The denver data were not readly available in a structured format like the case in toronto and new york. Therefor I had to obtain the data and put into a structred format and into a single csv file. This took the most time in the coding process

In [3]:

header=denver_data.iloc[0]
denver_data=denver_data[1:]
df=denver_data.rename(columns=header)


In [4]:
df.columns

Index(['Borough', 'Neighborhood', 'Latitude', 'Longitude'], dtype='object')

I redefined the dataframe to another and replaced the first row as a header. The reason for that is to avoid problems with the code later on.

In [5]:
latitude = float('39.73715')
longitude = float('-104.989174')
print('The geograpical coordinate of Denver are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Denver are 39.73715, -104.989174.


Obtaining the geographical location of Denver for the GET requests in foursquare

In [6]:
CLIENT_ID = 'OJ4HUXL1DFTO1JBWCJHVKOATZ4N1IKVB5ICSMCQPX4SOUUGH' # your Foursquare ID
CLIENT_SECRET = '5R4CYRHLWW0QXE1OYGO4GW5HF1WXN13DDACV1PTRYBS3BR1D' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OJ4HUXL1DFTO1JBWCJHVKOATZ4N1IKVB5ICSMCQPX4SOUUGH
CLIENT_SECRET:5R4CYRHLWW0QXE1OYGO4GW5HF1WXN13DDACV1PTRYBS3BR1D


In [7]:
df.loc[1, 'Neighborhood']

'Baker'

In [8]:
neighborhood_latitude = df.loc[1, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[1, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[1, 'Neighborhood'] # neighborhood name

radius=500

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Baker are 39.723919, -104.99515.


Choosing the first neighborhood (Baker) to explore the venues

In [9]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [10]:
results = requests.get(url).json()

In [11]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [29]:
venues = results['response']['groups'][0]['items']
    
nearby_venues =pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Zandi K Denver Central,Salon / Barbershop,39.725550,-104.997440
1,Trade,Gay Bar,39.723946,-104.998761
2,Santiago's Mexican Restaurant,Mexican Restaurant,39.725478,-104.998854
3,El Taco De Mexico,Mexican Restaurant,39.727642,-104.998277
4,space Gallery,Art Gallery,39.722743,-104.998264
5,Black Sky Brewery,Brewery,39.723932,-104.998366
6,Mile High Vienna Stand,Hot Dog Joint,39.721108,-104.998411
7,LaMar's Donuts and Coffee,Donut Shop,39.725538,-104.999581
8,Thai Curry,Asian Restaurant,39.725457,-104.999862
9,7-Eleven,Convenience Store,39.721188,-104.999248


Notice that there are 13 venues in Baker and a gym is not one of them. Baker could be a good candidate for opening a gym. However, I can not say for certian without explore all neighborhoods in Denver.

In [13]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

13 venues were returned by Foursquare.


In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

We use this loop to explore all venues in Denver's neighborhoods.

In [32]:
denver_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )


Baker
Capitol Hill
Central Business District
Cherry Creek
Cheesman Park
City Park
Congress Park
City Park West


KeyError: 'groups'

In [16]:
print(denver_venues.shape)
denver_venues.head()

(999, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Baker,39.723919,-104.99515,Zandi K Denver Central,39.725550,-104.997440,Salon / Barbershop
1,Baker,39.723919,-104.99515,Trade,39.723946,-104.998761,Gay Bar
2,Baker,39.723919,-104.99515,Santiago's Mexican Restaurant,39.725478,-104.998854,Mexican Restaurant
3,Baker,39.723919,-104.99515,El Taco De Mexico,39.727642,-104.998277,Mexican Restaurant
4,Baker,39.723919,-104.99515,space Gallery,39.722743,-104.998264,Art Gallery


In [17]:
denver_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Athmar Park,3,3,3,3,3,3
Auraria,11,11,11,11,11,11
Baker,13,13,13,13,13,13
Barnum,4,4,4,4,4,4
Barnum West,3,3,3,3,3,3
Bear Valley,4,4,4,4,4,4
Belcaro,3,3,3,3,3,3
Berkeley,18,18,18,18,18,18
Capitol Hill,56,56,56,56,56,56


We count the number of venues in each neighborhood and determined that Union Station has the most venues.

In [18]:
# one hot encoding
denver_onehot = pd.get_dummies(denver_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
denver_onehot['Neighborhood'] = denver_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [denver_onehot.columns[-1]] + list(denver_onehot.columns[:-1])
denver_onehot = denver_onehot[fixed_columns]

denver_onehot.head()

,Zoo Exhibit,ATM,Accessories Store,Alternative Healer,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Bike Rental / Bike Share,Bike Trail,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Diner,Disc Golf,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Event Space,Exhibit,Factory,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pizza Place,Playground,Pool,Pool Hall,Post Office,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Shipping Store,Shopping Mall,Ski Lodge,Ski Shop,Smoke Shop,Snack Place,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Food Gathering,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Track,Trade School,Train Station,Tree,Vacation Rental,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Baker,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Baker,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [33]:
denver_grouped = denver_onehot.groupby('Neighborhood').mean().reset_index()
denver_grouped

,Neighborhood,Zoo Exhibit,ATM,Accessories Store,Alternative Healer,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Bike Rental / Bike Share,Bike Trail,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Diner,Disc Golf,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Event Space,Exhibit,Factory,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pizza Place,Playground,Pool,Pool Hall,Post Office,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Shipping Store,Shopping Mall,Ski Lodge,Ski Shop,Smoke Shop,Snack Place,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Food Gathering,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Track,Trade School,Train Station,Tree,Vacation Rental,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Athmar Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0

Using one hot encoding to determine the frequency of venue. We noticed that there are a lot of neighborhoods that are missing a gym. Most of them could be a good candidate to open up.

In [20]:
num_top_venues = 5

for hood in denver_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = denver_grouped[denver_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Athmar Park----
                       venue  freq
0  Cajun / Creole Restaurant  0.33
1                       Park  0.33
2            Vacation Rental  0.33
3          Outdoor Sculpture  0.00
4          Mobile Phone Shop  0.00


----Auraria----
                  venue  freq
0           Coffee Shop  0.18
1           IT Services  0.09
2            Sports Bar  0.09
3           Gas Station  0.09
4  Fast Food Restaurant  0.09


----Baker----
                venue  freq
0  Mexican Restaurant  0.15
1         Art Gallery  0.15
2   Convenience Store  0.08
3    Asian Restaurant  0.08
4          Donut Shop  0.08


----Barnum----
                        venue  freq
0           Convenience Store  0.25
1  Construction & Landscaping  0.25
2                  Hookah Bar  0.25
3                      Market  0.25
4           Outdoor Sculpture  0.00


----Barnum West----
                  venue  freq
0          Home Service  0.33
1  Fast Food Restaurant  0.33
2     Electronics Store  0.33
3           Z

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In the next lines on code, we will cluster neighborhoods to their most common venues and try to pair neighborhoods with high number of venues. This will help us determine the most appropriate place for a gym since we want it in a popular area.

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = denver_grouped['Neighborhood']

for ind in np.arange(denver_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(denver_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Athmar Park,Park,Vacation Rental,Cajun / Creole Restaurant,Zoo,Dog Run,Flower Shop,Flea Market,Fast Food Restaurant,Farmers Market,Factory
1,Auraria,Coffee Shop,Theater,IT Services,Gas Station,Light Rail Station,Fast Food Restaurant,Salad Place,Brewery,Sports Bar,Hot Dog Joint
2,Baker,Mexican Restaurant,Art Gallery,Hot Dog Joint,Asian Restaurant,Dance Studio,Chinese Restaurant,Donut Shop,Salon / Barbershop,Brewery,Gay Bar
3,Barnum,Hookah Bar,Construction & Landscaping,Convenience Store,Market,Donut Shop,Food,Flower Shop,Flea Market,Fast Food Restaurant,Farmers Market
4,Barnum West,Home Service,Fast Food Restaurant,Electronics Store,Zoo,Dog Run,Food,Flower Shop,Flea Market,Farmers Market,Factory


In [23]:
# set number of clusters
kclusters = 5

denver_grouped_clustering = denver_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(denver_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 0, 0, 0, 2, 2, 0, 0, 0], dtype=int32)

In [24]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

denver_merged = df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
denver_merged = denver_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

denver_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Denver,Baker,39.723919,-104.99515,0.0,Mexican Restaurant,Art Gallery,Hot Dog Joint,Asian Restaurant,Dance Studio,Chinese Restaurant,Donut Shop,Salon / Barbershop,Brewery,Gay Bar
2,Denver,Capitol Hill,39.733333,-104.983333,0.0,Pizza Place,Nightclub,Coffee Shop,Sandwich Place,Gym,Hostel,Yoga Studio,Convenience Store,History Museum,Brewery
3,Denver,Central Business District,39.74001,-104.99202,0.0,Hotel,Art Gallery,Pub,Exhibit,Gym / Fitness Center,American Restaurant,Art Museum,Bar,Lounge,Marijuana Dispensary
4,Denver,Cherry Creek,39.720967,-104.949804,0.0,Cosmetics Shop,Coffee Shop,Mexican Restaurant,Furniture / Home Store,Asian Restaurant,American Restaurant,Restaurant,Pizza Place,New American Restaurant,Optical Shop
5,Denver,Cheesman Park,39.732778,-104.965833,0.0,Botanical Garden,Garden,Café,Park,Gift Shop,Track,Sports Club,Dance Studio,Dry Cleaner,Flower Shop


In [25]:
denver_merged.loc[denver_merged['Cluster Labels'] == 0, denver_merged.columns[[1] + list(range(5, denver_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Baker,Mexican Restaurant,Art Gallery,Hot Dog Joint,Asian Restaurant,Dance Studio,Chinese Restaurant,Donut Shop,Salon / Barbershop,Brewery,Gay Bar
2,Capitol Hill,Pizza Place,Nightclub,Coffee Shop,Sandwich Place,Gym,Hostel,Yoga Studio,Convenience Store,History Museum,Brewery
3,Central Business District,Hotel,Art Gallery,Pub,Exhibit,Gym / Fitness Center,American Restaurant,Art Museum,Bar,Lounge,Marijuana Dispensary
4,Cherry Creek,Cosmetics Shop,Coffee Shop,Mexican Restaurant,Furniture / Home Store,Asian Restaurant,American Restaurant,Restaurant,Pizza Place,New American Restaurant,Optical Shop
5,Cheesman Park,Botanical Garden,Garden,Café,Park,Gift Shop,Track,Sports Club,Dance Studio,Dry Cleaner,Flower Shop
6,City Park,Zoo Exhibit,American Restaurant,Exhibit,Gift Shop,Park,Track,Asian Restaurant,Lake,Outdoor Sculpture,Playground
7,Congress Park,Pizza Place,Food Truck,New American Restaurant,Ice Cream Shop,Burger Joint,Arts & Crafts Store,Tapas Restaurant,Flea Market,Fast Food Restaurant,Farmers Market
8,City Park West,Brewery,Pizza Place,Breakfast Spot,Mexican Restaurant,Thai Restaurant,Coffee Shop,American Restaurant,Pharmacy,Café,Sandwich Place
9,Civic Center,Hotel,Art Museum,Exhibit,Pub,Bar,Marijuana Dispensary,Burger Joint,Breakfast Spot,Dance Studio,Salad Place
10,Country Club,Golf Course,Gym / Fitness Center,Shipping Store,Sandwich Place,Salon / Barbershop,Discount Store,Farmers Market,Factory,Exhibit,Event Space


In [26]:
denver_merged.loc[denver_merged['Cluster Labels'] == 1, denver_merged.columns[[1] + list(range(5, denver_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Indian Creek,Park,Zoo,Dive Bar,Flower Shop,Flea Market,Fast Food Restaurant,Farmers Market,Factory,Exhibit,Event Space


In [27]:
denver_merged.loc[denver_merged['Cluster Labels'] == 3, denver_merged.columns[[1] + list(range(5, denver_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Windsor,Grocery Store,Flower Shop,Food Court,Food,Flea Market,Fast Food Restaurant,Farmers Market,Factory,Exhibit,Event Space


In [28]:
denver_merged.loc[denver_merged['Cluster Labels'] == 4, denver_merged.columns[[1] + list(range(5, denver_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,North Park Hill,Gym,Zoo,Dog Run,Food,Flower Shop,Flea Market,Fast Food Restaurant,Farmers Market,Factory,Exhibit
58,Hampden South,Garden,Bar,Gym,Zoo,Donut Shop,Food,Flower Shop,Flea Market,Fast Food Restaurant,Farmers Market


Looking at the first cluster, it appears that the best place for a gym is Spear or Lowry neighborhoods, because there are no gyms in those areas and they are close to union station one of the most popular attractions in Denver.